Using materials from course MTAT.03.227 Machine Learning

In [1]:
import numpy as np
import keras
from scipy import io

Using TensorFlow backend.
C:\Users\rasmusm\AppData\Local\Continuum\anaconda3\envs\neurovorgud\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rasmusm\AppData\Local\Continuum\anaconda3\envs\neurovorgud\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rasmusm\AppData\Local\Continuum\anaconda3\envs\neurovorgud\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  

In [2]:
mt = io.loadmat('6845348/images_natimg2800_all.mat')

In [3]:
imgs = mt['imgs']

In [4]:
#divided into 11 classes
classes = io.loadmat('stringer-pachitariu-et-al-2018b/classes/stimuli_class_assignment_confident.mat')

In [5]:
train = []
for i in range(2800):
    train.append(imgs[:,:,i])

In [6]:
train = np.asarray(train)

In [7]:
y_train = np.asarray(classes['class_assignment'][0])

Train test split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, y_train, test_size=0.12, random_state=0)

In [9]:
classes['class_assignment'][0]

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [10]:
y_train

array([ 0,  4,  6, ...,  2,  0, 11], dtype=uint8)

Standardize the inputs

In [12]:
mu = X_train.mean(axis=(0,1,2))
std = X_train.std(axis=(0,1,2))
X_train_norm = (X_train - mu)/std
X_test_norm = (X_test - mu)/std

Creating a convolutional neural network

In [16]:
from keras.layers import Input, Conv2D, Activation, Flatten, Dense, MaxPooling2D, BatchNormalization, Dropout, Reshape

x = Input(shape=(68, 270))
rs = Reshape((68, 270, 1))(x)
c1 = Conv2D(32, (3, 3), strides=(1, 1), padding="same")(rs)
b1 = BatchNormalization()(c1)
a1 = Activation('relu')(b1)
c2 = Conv2D(32, (3, 3), strides=(1, 1), padding="valid")(a1)
b2 = BatchNormalization()(c2)
a2 = Activation('relu')(b2)
p2 = MaxPooling2D(pool_size=2)(a2)
d2 = Dropout(rate=0.25)(p2)
f2 = Flatten()(d2)
h3 = Dense(100)(f2)
b3 = BatchNormalization()(h3)
a3 = Activation('relu')(b3)
d3 = Dropout(rate=0.5)(a3)
z = Dense(12)(d3)
p = Activation('softmax')(z)

In [17]:
from keras.models import Model
from keras.optimizers import Adam

model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 68, 270)           0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 68, 270, 1)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 68, 270, 32)       320       
_________________________________________________________________
batch_normalization_4 (Batch (None, 68, 270, 32)       128       
_________________________________________________________________
activation_5 (Activation)    (None, 68, 270, 32)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 66, 268, 32)       9248      
_________________________________________________________________
batch_normalization_5 (Batch (None, 66, 268, 32)       128 

In [18]:
history = model.fit(X_train_norm, y_train, batch_size=64, epochs=5, validation_split=0.04)

Train on 2365 samples, validate on 99 samples
Epoch 1/5
2365/2365 [==============================] - 526s 222ms/step - loss: 2.7235 - accuracy: 0.1712 - val_loss: 2.6869 - val_accuracy: 0.1414
Epoch 2/5
2365/2365 [==============================] - 523s 221ms/step - loss: 1.7010 - accuracy: 0.4406 - val_loss: 2.5925 - val_accuracy: 0.0909
Epoch 3/5
2365/2365 [==============================] - 521s 220ms/step - loss: 1.0699 - accuracy: 0.6778 - val_loss: 2.8252 - val_accuracy: 0.1212
Epoch 4/5
2365/2365 [==============================] - 524s 222ms/step - loss: 0.6135 - accuracy: 0.8715 - val_loss: 2.5897 - val_accuracy: 0.1212
Epoch 5/5
2365/2365 [==============================] - 526s 223ms/step - loss: 0.4111 - accuracy: 0.9345 - val_loss: 2.4794 - val_accuracy: 0.1111


Evaluate on test

In [19]:
model.evaluate(X_test_norm, y_test)

336/336 [==============================] - 19s 56ms/step


[2.4774220898037864, 0.1369047611951828]

Evaluate on train

In [20]:
model.evaluate(X_train_norm, y_train)

2464/2464 [==============================] - 134s 55ms/step


[1.4992056171615402, 0.6160714030265808]